In [1]:
import pandas as pd
import requests
import json
import os
from os import path

from cryptography.hazmat.backends import default_backend
from datetime import datetime
from prefect import task, flow

import os, sys
parent_dir = os.path.abspath('.')
from configuration.config import Config
config = Config(parent_dir, 'creds.json')
pass


/Users/kcbigring/workspace/kcbigring/lgc/codez/prefect-legacy/configuration/config.py:4: RuntimeWarning: coroutine 'Block.load' was never awaited
  Secret.load("github-access-token")


In [2]:
# Task 1: Authentication
@task
def authenticate():
  GetTokenURL = "https://clmapi.landgorilla.com/api/token"
  USER = "dustinh@legacyg.com"
  PW = "Index@2043!"
  body = {'api_name': 'clm'}
  headers = {
    'USER': USER,
    'PASSWORD': PW
    }
  response = requests.get(GetTokenURL, headers=headers, json=body)
  json_data = response.json()
  if response.status_code == 200:
    json_data = response.json()
    api_token = json_data.get('token')
    

    if api_token:
        your_variable_name = api_token
        print(f"API Token: {api_token}")
    else:
        print("No API token found in the response JSON.")
  else:
     print("Request was not successful. Status code:", response.status_code)
  return json_data.get('token')


In [3]:
# Task 2: Get most recent ID value in order to pull the most recent report 
@task
def get_most_recent_id_value(api_token):
    ReportURL = f"https://clmapi.landgorilla.com/api/clm/pipelineReport"
    body = {'api_name': 'clm'}
    headers = {
        'Authorization': f'Bearer {api_token}'
        }
        
    ReportResponse = requests.get(ReportURL, headers=headers, json= body)
    print(ReportResponse.text)
    ReportData = ReportResponse.json()
    # Function to convert date string to a comparable format (e.g., MM/DD/YYYY)
    def convert_date(date_string):
        # Assuming the date format is MM/DD/YYYY
        return datetime.strptime(date_string, '%m/%d/%Y')
    
    # Get the dictionary with the most recent reportdate for a specific report name
    report_name = 'Maturity Dates'
    
    latest_report = max((data for data in ReportData if data['name'] == report_name), key=lambda x: int(x['id']), default=None)
    latest_id_value = latest_report.get('id')
    return latest_id_value


In [4]:
# Task 3: Pull Report Using Latest ID Value and API Token
@task
def pull_maturity_dates_report(latest_id_value, api_token):
    ActualReportURL = f"https://clmapi.landgorilla.com/api/clm/pipelineReport/{latest_id_value}"
    headers = {
        'Authorization': f'Bearer {api_token}'
        }
    
    ActualReportResponse = requests.get(ActualReportURL, headers=headers)
    ReportData = ActualReportResponse.json()
    data_list = ReportData['data']['Current Report']['data']
    df = pd.DataFrame(data_list)
    return df


In [5]:
# Task 4: Transform Data
import os, sys
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from transform.land_gorilla_maturity_date_transformer import LandGorillaMaturityDateTransformer

@task
def transform_data(df):
    transformer = LandGorillaMaturityDateTransformer(df)
    transformer.transform_data()
    return transformer.df

In [6]:
# Task 5: Export File
import os, sys
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from load.land_gorilla_maturity_date_loader import LandGorillaMaturityDateLoader

@task
def export_file(df_write):
    config = Config('../creds.json')
    username = config.username
    account = config.account
    password = config.password
    database = config.database
    schema = config.schema
    warehouse = config.warehouse

    # this is a public key so can live in github
    script_path = os.path.dirname(os.path.abspath("__file__"))
    key_path = path.join(script_path, 'keys', config.private_key)

    loader = LandGorillaMaturityDateLoader(username, account, password, database, schema, warehouse, key_path)
    loader.write_table(df_write)

In [7]:
@flow (log_prints= True)
def land_gorilla_maturity_dates_report() -> None:
    api_token = authenticate()
    most_recent_id = get_most_recent_id_value(api_token)
    most_recent_maturity_dates_report = pull_maturity_dates_report(most_recent_id, api_token)
    transformed_most_recent_maturity_dates_report = transform_data(most_recent_maturity_dates_report)
    temp = transformed_most_recent_maturity_dates_report[["loan_number", "borrower"]]
    export_file(temp)

if __name__ == "__main__":
    land_gorilla_maturity_dates_report()

10:22:32.958 | INFO    | prefect.engine - Created flow run 'spry-kestrel' for flow 'land-gorilla-maturity-dates-report'

10:22:32.965 | INFO    | Flow run 'spry-kestrel' - View at https://app.prefect.cloud/account/be0b832f-4c96-44af-8996-6408d20903a1/workspace/70b125ce-298b-4133-83d1-21a2d99ae339/flow-runs/flow-run/d05f0816-172d-4d5f-9789-0d94568c8f1d

10:22:33.305 | INFO    | Flow run 'spry-kestrel' - Created task run 'authenticate-0' for task 'authenticate'

10:22:33.307 | INFO    | Flow run 'spry-kestrel' - Executing 'authenticate-0' immediately...

10:22:34.126 | INFO    | Task run 'authenticate-0' - API Token: eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJMYW5kR29yaWxsYSIsImF1ZCI6Imh0dHBzOi8vc3lzdGVtLmxhbmRnb3JpbGxhLmNvbS8iLCJpYXQiOjE3MTg0Njg1NTQsIm5iZiI6MTcxODQ2ODU1NCwiZXhwIjoxNzE4NDcyMTU0LCJkYXRhIjp7ImlkIjoic0E4U1FXTGd0YlhmV3Z0d2pqWnIzMzVTRlhaaWJ4S2RheDBrR3FhQTFTWU9meU9tUFNLOU4ycnQ3Zk44R3R2QTlCejBseXBSS3dMS0tEekI0ODZ2NkxuRkdFUlRSNWZDZ2pTYUVNR05peGo2cUdWZ0lwMUZTQWtKUy9NL1JZbjEiLCJ0eXBlIjoiZEFUMTN1S3VLWjRGUjI4aWtYUlNKZ2ZSYmFOVmRueHVLSGQ5V05XK0s0bmg2QzQ4N3dtSDZFdXRCSHVpU3o4Wlh1dm1xNVV0a2syOGhOUENkemVrUGZVVWVNazdsYytsTVRvL3ZUOGljSTF4Q05zTjFvcEVCajdoZTZmM3phVEM4aVdmSDJTVTMxc2YwTUNiRTRhTjB3PT0ifX0.msxIiG8tzRILCYolE3AFfH9pgdxyWEIVKSbKRITBcSY

...

10:22:37.387 | INFO    | Task run 'transform_data-0' - Finished in state Completed()

10:22:37.575 | INFO    | Flow run 'spry-kestrel' - Created task run 'export_file-0' for task 'export_file'

10:22:37.577 | INFO    | Flow run 'spry-kestrel' - Executing 'export_file-0' immediately...

10:22:41.538 | INFO    | Task run 'export_file-0' - Finished in state Completed()

10:22:41.692 | INFO    | Flow run 'spry-kestrel' - Finished in state Completed('All states completed.')